# 서울시 교통사고 시각화 

In [167]:
import pandas as pd

In [168]:
traffic_Seoul = pd.read_csv("../Data/newSeoul_2005_2019.csv")
traffic_Seoul.head()


,년도,월,자치구명,발생건수,사망자수,부상자수
0,2005,1,종로구,93,2,138
1,2005,2,종로구,84,3,125
2,2005,3,종로구,117,0,142
3,2005,4,종로구,138,2,212
4,2005,5,종로구,145,2,207


In [169]:
# 2019년 데이터만 추출하기 
traffic_Seoul_2019 = traffic_Seoul[traffic_Seoul['년도']==2019]

In [170]:
# index 정렬
traffic_Seoul_2019.reset_index(drop =True ,inplace=True)
traffic_Seoul_2019

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2019,1,종로구,87,1,125
1,2019,2,종로구,66,1,84
2,2019,3,종로구,87,2,122
3,2019,4,종로구,85,0,131
4,2019,5,종로구,112,1,158
...,...,...,...,...,...,...
295,2019,8,강동구,127,2,175
296,2019,9,강동구,98,2,137
297,2019,10,강동구,108,1,144
298,2019,11,강동구,146,0,199


In [171]:
# 자치구별 발생건수, 사망자수 , 부상자수 집계하기 
traffic_analysis = pd.pivot_table(traffic_Seoul_2019[['자치구명','발생건수','사망자수','부상자수']],
index='자치구명',aggfunc=sum)
traffic_analysis

,발생건수,부상자수,사망자수
자치구명,,,
강남구,3722,5182,14
강동구,1414,1910,11
강북구,1277,1706,7
강서구,1829,2491,20
관악구,1363,1755,10
광진구,973,1316,3
구로구,1529,2096,13
금천구,920,1353,9
노원구,1703,2302,8


In [172]:
# 위도 경도 불러오기
# 윈도우에서 만든 csv encoding = euc-kr ,ios = utf-8
seoul_limit = pd.read_csv("../Data/seoul.csv",encoding='euc-kr')
seoul_limit.columns = ['자치구명','lon','lat']
seoul_limit.head()

,자치구명,lon,lat
0,강남구,127.0475,37.51731
1,강동구,127.1238,37.53013
2,강북구,127.0255,37.63975
3,관악구,126.9515,37.47834
4,구로구,126.8875,37.49547


In [173]:
# traffic_analysis ,병합 seoul_limit
data_result= pd.merge(traffic_analysis,seoul_limit,on='자치구명' )
data_result.head()

,자치구명,발생건수,부상자수,사망자수,lon,lat
0,강남구,3722,5182,14,127.0475,37.51731
1,강동구,1414,1910,11,127.1238,37.53013
2,강북구,1277,1706,7,127.0255,37.63975
3,강서구,1829,2491,20,126.8496,37.55094
4,관악구,1363,1755,10,126.9515,37.47834


In [174]:
data_result.tail()

,자치구명,발생건수,부상자수,사망자수,lon,lat
20,용산구,1187,1614,10,126.9904,37.53254
21,은평구,1119,1464,7,126.9292,37.60278
22,종로구,1133,1559,6,126.9793,37.57323
23,중구,1184,1615,5,127.0032,37.55273
24,중랑구,1620,2122,12,127.0928,37.60652


In [175]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     int64  
 2   부상자수    25 non-null     int64  
 3   사망자수    25 non-null     int64  
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.4+ KB


In [176]:
# folium 을 사용하기 위해 int를 float로 변환 
data_result[["발생건수","부상자수",'사망자수']] = data_result[["발생건수","부상자수",'사망자수']].astype('float')


In [177]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     float64
 2   부상자수    25 non-null     float64
 3   사망자수    25 non-null     float64
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.4+ KB


### 자치구의 구청을 표시하기 

In [178]:
import folium

In [179]:
# 위도 경도 순서 바꿔주기 
seoul_limit.head()

,자치구명,lon,lat
0,강남구,127.0475,37.51731
1,강동구,127.1238,37.53013
2,강북구,127.0255,37.63975
3,관악구,126.9515,37.47834
4,구로구,126.8875,37.49547


In [180]:
seoul_limit = seoul_limit[['자치구명','lat','lon']]

In [181]:
seoul_limit.head()

,자치구명,lat,lon
0,강남구,37.51731,127.0475
1,강동구,37.53013,127.1238
2,강북구,37.63975,127.0255
3,관악구,37.47834,126.9515
4,구로구,37.49547,126.8875


In [182]:
# seoul csv 를 이용하여 구청 표시하기 
map = folium.Map(
    # 처음 지도를 볼 때 중심이 되는 위치
    location=[37.5502,126.982]  # 남산서울타워쪽 
)
# 구청 위치 정보를 Marker으로 표시 
for name,lat, lng in zip(seoul_limit.자치구명,seoul_limit.lat, seoul_limit.lon):
    folium.Marker([lat,lng],
    color = 'red',
    popup=name).add_to(map)
    # popup 하면 이름 출력 

# circle marker를 통해 교통사고 발생건수 표시 
for name,lat, lng, case in zip(seoul_limit.자치구명,seoul_limit.lat, seoul_limit.lon,data_result.발생건수):
    folium.CircleMarker(
        [lat,lng],
        radius= case / 100, # 원의 반지름    # / 100 은 hyper parameter로 내가 직접 세팅하는 값
        color ='limegreen',
        fill =True,
        fill_color = 'yellow',
        fill_opacity = 0.5,  # 원채우기 투명도
        popup=name,
        ).add_to(map)

map

In [183]:
data_result.sort_values(by="사망자수",ascending=False,inplace= True)

In [190]:
data_result = data_result[['자치구명','발생건수','부상자수','사망자수','lat','lon']]
data_result

,자치구명,발생건수,부상자수,사망자수,lat,lon
3,강서구,1829.0,2491.0,20.0,37.55094,126.8496
10,동대문구,1565.0,1988.0,15.0,37.57452,127.0396
0,강남구,3722.0,5182.0,14.0,37.51731,127.0475
14,서초구,2418.0,3530.0,14.0,37.48400,127.0330
13,서대문구,1113.0,1609.0,14.0,37.57908,126.9368
17,송파구,2892.0,4003.0,13.0,37.51448,127.1059
15,성동구,1091.0,1439.0,13.0,37.56346,127.0368
6,구로구,1529.0,2096.0,13.0,37.49547,126.8875
24,중랑구,1620.0,2122.0,12.0,37.60652,127.0928
1,강동구,1414.0,1910.0,11.0,37.53013,127.1238


In [192]:
data_result.reset_index(drop =True ,inplace=True)

In [193]:
data_result.head()

,자치구명,발생건수,부상자수,사망자수,lat,lon
0,강서구,1829.0,2491.0,20.0,37.55094,126.8496
1,동대문구,1565.0,1988.0,15.0,37.57452,127.0396
2,강남구,3722.0,5182.0,14.0,37.51731,127.0475
3,서초구,2418.0,3530.0,14.0,37.48400,127.0330
4,서대문구,1113.0,1609.0,14.0,37.57908,126.9368


---
# 자치구별 사망자수 표시하기

In [210]:
# seoul csv 를 이용하여 구청 표시하기 
map = folium.Map(
    # 처음 지도를 볼 때 중심이 되는 위치
    location=[37.5502,126.982]  # 남산서울타워쪽 
)
# 구청 위치 정보를 Marker으로 표시 
for name,lat, lng in zip(data_result.자치구명,data_result.lat, data_result.lon):
    folium.Marker([lat,lng],
    popup=name).add_to(map)
    # popup 하면 이름 출력 

# circle marker를 통해 교통사고 발생건수 표시 
for name,lat, lng, case in zip(data_result.자치구명,data_result.lat, data_result.lon,data_result.사망자수):
    folium.CircleMarker(
        [lat,lng],
        #data = data_result['사망자수'],
        radius= case *2, # 원의 반지름    # / 100 은 hyper parameter로 내가 직접 세팅하는 값
        color ='red',
        fill =True,
        fill_color = 'red',
        fill_opacity = 0.3,  # 원채우기 투명도
        popup=name,
        ).add_to(map)

# 사망자수가 제일 높은 곳 표시 
folium.CircleMarker(
        location = [data_result.iloc[0]['lat'],data_result.iloc[0]['lon']],
        radius= 40 , # 원의 반지름    # / 100 은 hyper parameter로 내가 직접 세팅하는 값
        color ='blue',
        fill =True,
        fill_color = 'red',
        fill_opacity = 0.3,  # 원채우기 투명도
        popup=name,
        ).add_to(map)
# 사망자수가 제일 높은 곳 표시 
folium.CircleMarker(
        location = [data_result.iloc[24]['lat'],data_result.iloc[24]['lon']],
        radius= 6 , # 원의 반지름    # / 100 은 hyper parameter로 내가 직접 세팅하는 값
        color ='blue',
        fill =True,
        fill_color = 'red',
        fill_opacity = 0.3,  # 원채우기 투명도
        popup=name,
        ).add_to(map)

map

In [209]:
#data_result['사망자수'].max()
data_result['사망자수'].min()

3.0

In [202]:
data_result['lat'][0]

37.55094

In [ ]:
data_result